## Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib as mpl

In [3]:
os.getcwd()

'/Users/ricardovelloso/git-projects/binder-framework'